In [1]:
!pip install wandb==0.16.0

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.utils import plot_model
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
class_to_idx = {
    '협박 대화' : 0,
    '갈취 대화' : 1,
    '직장 내 괴롭힘 대화' : 2,
    '기타 괴롭힘 대화' : 3,
    '일반 대화' : 4
}

In [4]:
# Load data
train_df = pd.read_csv('custom_train.csv')
test_df = pd.read_csv('custom_test.csv')

# Prepare the data
max_words = 10000
max_len = 500
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(train_df['conversation'].values)

X_train = tokenizer.texts_to_sequences(train_df['conversation'].values)
X_train = pad_sequences(X_train, maxlen=max_len)

train_df['label'] = train_df['class'].apply(lambda x: class_to_idx[x])

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df['label'])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(np.unique(y_train)))
y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(np.unique(y_val)))

X_test = tokenizer.texts_to_sequences(test_df['conversation'].values)
X_test = pad_sequences(X_test, maxlen=max_len)

In [6]:
wandb.login(key = '개인키')

wandb: Currently logged in as: janghyeon06 (janghyeon). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [7]:
sweep_config = {
    "name": "sweep_test_nlp",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

In [8]:
def train():
    default_config = {
        "vocab" : 1000,
        "embeddings" : 64,
        "units_1" : 128,
        "units_2" : 128,
        "units_3" : 64,
        "class_num" : 5,
        "learning_rate" : 0.005,
        "optimizer" : "adam",
        "loss" : "categorical_crossentropy",
        "metrics" : ["accuracy"],
        "epoch" : 5,
        "batch_size" : 32
    }
    import os

    # 환경 변수 설정
    os.environ['WANDB_AGENT_DISABLE_FLAPPING'] = 'true'

    wandb.init(config = default_config)
    config = wandb.config

    # Model

    model = Sequential()
    model.add(Embedding(config.vocab, config.units_1, input_length=max_len))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(config.units_2, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(config.units_3, activation='relu'))
    model.add(Dense(len(np.unique(train_df['label'])), activation='softmax'))

    # 머신 러닝 학습때 여러가지 optimzier를 사용할 경우나 learning rate를 조절할 경우에는 아래와 같은 형태의 코드를 응용합니다.

    if config.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)
    
    model.compile(optimizer = optimizer,
                  loss = config.loss,
                  metrics = config.metrics)

    # WandbCallback 함수는 후술합니다.
    
    EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
    
    model.fit(X_train, y_train,
              epochs = config.epoch,
              batch_size = config.batch_size,
              validation_split=0.2,
              callbacks = [EarlyStopping, WandbCallback()])
    
    validation_loss, validation_accuracy = model.evaluate(X_val, y_val, verbose=2)
    
    # wandb.log 함수 안에 기록하고 싶은 정보를 담습니다.
    
    wandb.log({"Validation Accuracy Rate: " : round(validation_accuracy * 100, 2),
               "Validation Error Rate: " : round((1 - validation_accuracy) * 100, 2)})

In [ ]:
# entity와 project에 본인의 아이디와 프로젝트명을 입력하세요

sweep_id = wandb.sweep(sweep_config,
                       entity = 'janghyeon',
                       project = 'DLTHON1')

# run the sweep
wandb.agent(sweep_id,
            function=train,
            count=10)

Create sweep with ID: 3dvjoktg
Sweep URL: https://wandb.ai/janghyeon/DLTHON1/sweeps/3dvjoktg


wandb: Agent Starting Run: p2sdwiro with config:
wandb: 	epoch: 6
wandb: 	learning_rate: 0.07732787456900941


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/6
101/101 [==============================] - 170s 2s/step - loss: 1.9177 - accuracy: 0.2061 - val_loss: 1.6053 - val_accuracy: 0.2240
INFO:tensorflow:Assets written to: /aiffel/aiffel/wandb/run-20240626_020916-p2sdwiro/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/aiffel/wandb/run-20240626_020916-p2sdwiro/files/model-best)... Done. 0.0s


...

Validation Accuracy Rate:,▁
Validation Error Rate:,▁
accuracy,▁▆▅█▅
epoch,▁▃▅▆█
loss,█▁▁▁▁
val_accuracy,███▁▁
val_loss,█▄▁▁▄
Validation Accuracy Rate:,21.49
Validation Error Rate:,78.51
accuracy,0.21968
best_epoch,2


wandb: Agent Starting Run: oynedlrw with config:
wandb: 	epoch: 7
wandb: 	learning_rate: 0.09617993341237918


Epoch 1/7
101/101 [==============================] - 173s 2s/step - loss: 2.0139 - accuracy: 0.2169 - val_loss: 1.6154 - val_accuracy: 0.2240
INFO:tensorflow:Assets written to: /aiffel/aiffel/wandb/run-20240626_031640-oynedlrw/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/aiffel/wandb/run-20240626_031640-oynedlrw/files/model-best)... Done. 0.0s


Epoch 2/7
  7/101 [=>............................] - ETA: 2:33 - loss: 1.6038 - accuracy: 0.2455